In [ ]:
import numpy as np
#座標轉換
def calculate_rotation_angles(x_new, z_new):
    # Normalize the vectors
    x_new = x_new / np.linalg.norm(x_new)
    z_new = z_new / np.linalg.norm(z_new)
    
    # Calculate y_new as the cross product of z_new and x_new
    y_new = np.cross(z_new, x_new)
    
    # Ensure y_new is also normalized
    y_new = y_new / np.linalg.norm(y_new)
    
    # Construct the rotation matrix
    rotation_matrix = np.array([x_new, y_new, z_new]).T
    
    # Calculate rotation angles
    rx = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
    ry = np.arcsin(-rotation_matrix[2, 0])
    rz = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
    
    # Convert angles to degrees
    rx_degrees = np.degrees(rx)
    ry_degrees = np.degrees(ry)
    rz_degrees = np.degrees(rz)
    
    # Adjust angles to be in the range [0, 360]
    if rx_degrees < 0:
        rx_degrees += 360
    if ry_degrees < 0:
        ry_degrees += 360
    if rz_degrees < 0:
        rz_degrees += 360
    
    return rx_degrees, ry_degrees, rz_degrees

# Example usage
# x_new = np.array([0, 1, 0])  # Example new x axis vector
# z_new = np.array([1, 0, 0])  # Example new z axis vector
# rx_degrees, ry_degrees, rz_degrees = calculate_rotation_angles(x_new, z_new)
# print(f"Rotation around X axis (rx): {rx_degrees} degrees")
# print(f"Rotation around Y axis (ry): {ry_degrees} degrees")
# print(f"Rotation around Z axis (rz): {rz_degrees} degrees")

In [ ]:
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln, gp_Vec
def Coordinate_conversion(path,name,is_save=True):
    points = []  # 创建一个空列表来存储解析后的reward值
    with open(path, "r") as file:
        for line in file:
            # 输出每一行内容，以确保正确读取
            # print("Line:", line)
            try:
                # 从每一行中解析出8个值
                face, trajc, x, y, z, rx, ry, rz = line.split()
                # 将解析后的值作为一个元组添加到rewards列表中
                points.append((face, trajc, x, y, z, rx, ry, rz))
            except ValueError:
                # 如果发生拆包错误，输出错误信息
                print("Error: Unable to unpack values from line:", line)

    max_value_a = max(int(point[0]) for point in points)
    # print("Maximum value of point[1]:", max_value_a)
    max_value_b = max(int(point[1]) for point in points)

    for a in range(1, max_value_a+1): 
        same_surface_points = [point for point in points if float(point[0]) ==  a]#提取同一面上的點
        print(len(same_surface_points))
        # print(same_surface_points)

        # same_surface_points=[point for point in points if point[0] == '2']#提取同一面上的點
        for b in range(1, max_value_b+1): 

            same_path_points = [point for point in same_surface_points if float(point[1]) ==  float(b)]#提取同一路徑上的點

            #same_path_points = [point for point in same_surface_points if point[1] == '1']#提取同一路徑上的點
            for i, point in enumerate(same_path_points):
                point = [float(item) for item in point]  # 将字符串转换为浮点数
                # print("Current Point:", point)
                
                point_current=np.array((point[2],point[3],point[4]))
                
                # print(point_end)
                # normal_vector_end=np.array((point[5],point[6],point[7]))
                z_vector=np.array((point[5],point[6],point[7]))
                # print(normal_vector_end)

                if i > 0:
                    next_point = [float(item) for item in same_path_points[i - 1]]  # 获取前一个 point 的信息

                    # print("Previous Point:", previous_point)
                    point_next=np.array((next_point[2],next_point[3],next_point[4]))
                    #計算x方向
                    x_vector=point_current-point_next
                    rx_degrees, ry_degrees, rz_degrees = calculate_rotation_angles(x_vector,z_vector)
                    # print(f"Rotation around X axis (rx): {rx_degrees} degrees")
                    # print(f"Rotation around Y axis (ry): {ry_degrees} degrees")
                    # print(f"Rotation around Z axis (rz): {rz_degrees} degrees")                
                else:
                    previous_point = [float(item) for item in same_path_points[i + 1]]
                    
                    # print("Previous Point:", previous_point)
                    point_previous=np.array((previous_point[2],previous_point[3],previous_point[4]))
                    #計算x方向
                    x_vector=point_previous-point_current
                    rx_degrees, ry_degrees, rz_degrees = calculate_rotation_angles(x_vector,z_vector)
                    # print(f"Rotation around X axis (rx): {rx_degrees} degrees")
                    # print(f"Rotation around Y axis (ry): {ry_degrees} degrees")
                    # print(f"Rotation around Z axis (rz): {rz_degrees} degrees")



                if is_save:
                    with open(name  + '.txt', 'a') as outfile:
                        outfile.write(str(a) + '\t' + str(b) +
                                    '\t' + format(point[2], ".2f") + '\t' + format(point[3], ".2f") + '\t' + format(point[4], ".2f") +
                                    '\t' + format(rx_degrees, ".2f") + '\t' + format(ry_degrees, ".2f") + '\t' + format(rz_degrees, ".2f") + '\n')

path=r"C:\alan\ITRI\ITRI-test\trajectory\simple3_0704_10.txt"
name="0709_rotation_angles"
Coordinate_conversion(path,name,is_save=True)